# Bollinger Bands strategy on EUR/GBP 1min data

In [1]:
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
import datetime

In [2]:
#Read data from onedrive
url ="https://onedrive.live.com/download?resid=4E16D53638D067A9%21339322&authkey=!AE2f-Z1jWB8rc3w"
eurgbp = pd.read_csv(url, parse_dates=True, index_col=0)

In [3]:
class BollingerStrategy(bt.Strategy):
    params = (
        ('period', 60),
        ('devfactor', 5),
    )

    def __init__(self):
        self.boll = bt.indicators.BollingerBands(period=self.p.period, devfactor=self.p.devfactor)

    def next(self):
        if not self.position:
            if self.data.close > self.boll.lines.top:
                self.sell()
            if self.data.close < self.boll.lines.bot:
                self.buy()

        else:
            if self.position.size > 0 and self.data.close > self.boll.lines.mid:
                self.close()
            if self.position.size < 0 and self.data.close < self.boll.lines.mid:
                self.close()
    
    def notify_order(self, order):
        if order.status == order.Completed:
            action = 'BUY' if order.isbuy() else 'SELL'
            print(f'{action} EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}')
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f'Trade PNL: {trade.pnl:.2f}')


In [4]:
#Avvio Cerebro e importo i dati
cerebro = bt.Cerebro(stdstats=False)
cerebro = bt.Cerebro(cheat_on_open=True) #per calcolare la size basata sulla open dopo (all in method)


bt_data = bt.feeds.PandasData(dataname=eurgbp, timeframe = bt.TimeFrame.Minutes)
cerebro.adddata(bt_data)
cerebro.addstrategy(BollingerStrategy)
cerebro.addsizer(bt.sizers.PercentSizer, percents=90)

#Aggiungo lo slippage
cerebro.broker.set_slippage_fixed(0.000125, slip_open=True, slip_match=True, slip_out=True) #ho messo 1.25 pips come spread (media di ICMarkets.eu)

#Aggiugni gli analyzers e gli observers
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.SQN)


results = cerebro.run()
firstStrat = results[0]

cerebro.plot(iplot=False, volume=False, width=50, height=20, style='candle')
plt.show()

BUY EXECUTED, Price: 0.86, Cost: 9004.86
SELL EXECUTED, Price: 0.86, Cost: 9004.86
Trade PNL: 7.95
BUY EXECUTED, Price: 0.86, Cost: 9008.56
SELL EXECUTED, Price: 0.86, Cost: 9008.56
Trade PNL: 7.73
BUY EXECUTED, Price: 0.86, Cost: 9015.73
SELL EXECUTED, Price: 0.86, Cost: 9015.73
Trade PNL: 1.05
BUY EXECUTED, Price: 0.86, Cost: 9016.78
SELL EXECUTED, Price: 0.86, Cost: 9016.78
Trade PNL: 5.35
BUY EXECUTED, Price: 0.86, Cost: 9018.98
SELL EXECUTED, Price: 0.86, Cost: 9018.98
Trade PNL: 10.92
BUY EXECUTED, Price: 0.86, Cost: 9031.96
SELL EXECUTED, Price: 0.86, Cost: 9031.96
Trade PNL: 1.15
BUY EXECUTED, Price: 0.86, Cost: 9032.26
SELL EXECUTED, Price: 0.86, Cost: 9032.26
Trade PNL: -7.36
BUY EXECUTED, Price: 0.86, Cost: 9025.12
SELL EXECUTED, Price: 0.86, Cost: 9025.12
Trade PNL: 0.95
SELL EXECUTED, Price: 0.86, Cost: -9023.55
BUY EXECUTED, Price: 0.86, Cost: -9023.55
Trade PNL: 1.79
BUY EXECUTED, Price: 0.86, Cost: 9030.33
SELL EXECUTED, Price: 0.86, Cost: 9030.33
Trade PNL: 4.75
BUY EX

In [5]:
for x in firstStrat.analyzers:
    x.print()

TradeAnalyzer:
  -----------------------------------------------------------------------------
  - total:
    - total: 144
    - open: 0
    - closed: 144
  -----------------------------------------------------------------------------
  - streak:
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - won:
      - current: 5
      - longest: 15
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - lost:
      - current: 0
      - longest: 4
  -----------------------------------------------------------------------------
  - pnl:
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - gross:
      - total: 564.294067736486
      - average: 3.9187088037255973
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - net:
      - total: 564.294067736486
      - average: 3.9187088037255973
  -----------------------------------------------------------------------------
  - 